In [800]:
import pathlib
from pathlib import Path

import numpy as np
import pandas as pd

#%load_ext lab_black

pd.options.display.max_colwidth = 250
pd.options.display.max_columns = 250

In [801]:
from google.colab import drive

drive.mount('/content/drive')

DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)
(DATA_DIR / "raw").mkdir(exist_ok=True)

!cp drive/MyDrive/comp_data/openhack/kanunum-nlp-doc-analysis-dataset.csv data/raw

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [802]:
df = pd.read_csv(Path(DATA_DIR) / "raw" / "kanunum-nlp-doc-analysis-dataset.csv")
df["kategori"] = df["kategori"].str.lower().str.strip()

num_class = df["kategori"].nunique()

In [799]:
#rega_no, Mükerrer no, sira no

| Kategori                                                               | İçerik Sayısı     | Hedef Değişkenler                                                                                            |
| :-------                                                               | :------------     | :---------------------------                                                                                              |
| [Kanun](#kategori-kanun)                                               | 514               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`,`mevzuat_tarihi`, `madde_sayisi`
| [Kanun Hükmünde Kararname](#kategori-kanun-hükmünde-kararname-khk)     | 91                | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `mevzuat_tarihi`, `madde_sayisi`
| [Resmi Gazete](#kategori-resmi-gazete)                                 | 546               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`
|
| [Komisyon Raporu](#kategori-komisyon-raporu)                           | 500               | `kategori`, `donem` ve `sira_no`                                                                       |
| [Genelge](#kategori-genelge)                                           | 385               | `kategori`, `mevzuat_tarihi` ve varsa `mevzuat_no` yoksa `belge_sayi` 
| [Cumhurbaşkanlığı Kararnamesi](#kategori-cumhurbaşkanlığı-kararnamesi) | 54                | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `madde_sayisi` ve varsa ` mevzuat_tarihi`
| [Tüzük](#kategori-tüzük)                                               | 82                |  `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `mevzuat_no`, `mevzuat_tarihi`, `madde_sayisi`
| [Yönetmelik](#kategori-yönetmelik)                                     | 597               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `kurum`, `madde_sayisi`       |
| [Tebliğ](#kategori-tebliğ)                                             | 623               | `kategori`, `rega_no`, `mukerrer_no`, `rega_tarihi`, `kurum` 
| [Özelge](#kategori-özelge)                                             | 750               | `kategori`, `mevzuat_tarihi`

In [797]:
#list(pd.Index(df["kategori"].unique()).difference(["komisyon raporu", "genelge", "özelge"]))

In [803]:
exclude_conf = {"mukerrer_no": ["komisyon raporu", "genelge", "özelge"],
 "sira_no": ['cumhurbaşkanlığı kararnamesi',
 'genelge',
 'kanun',
 'kanun hükmünde kararname',
 'resmi gazete',
 'tebliğ',
 'tüzük',
 'yönetmelik',
 'özelge'],
 "rega_no":  ["komisyon raporu", "genelge", "özelge"],
}

In [805]:
import re

def iter_pattern(string, pattern, group=2):

    collection = re.finditer(pattern, string)

    if not collection:
        raise Exception(f"The pattern {pattern} isn't found in {string}")

    for match in collection:
        try:
            match_ = match.group(group)
        except Exception as e:
            raise e
        else:
            break
    
    return match_
    

def extract_rega_no_pattern(row):
    pattern = "(sayı:)(\d+)"

    try:
      rega_no = int(iter_pattern(row, pattern, 2))#int(re.search(pattern, row).group(2))
    except Exception as e:
        
        try:
          pattern = "(\d+)(sayı)"
          rega_no = int(iter_pattern(row, pattern, 1))#int(re.search(pattern, row).group(1))
        
        except Exception as e:
            print(row, e)
            rega_no = None

    return rega_no

def extract_mukerrer_no_pattern(row):
    pattern = "(\d+)\. mükerrer"

    try:
      mukerrer_no = int(iter_pattern(row, pattern, 1))#int(re.search(pattern, row).group(2))
    except Exception as e:
      print(row, e)
      mukerrer_no = 0

    return mukerrer_no

In [806]:
def extract_sira_no_pattern(row):
    pattern = "(\d+) (e \| nci|e \| ci|e ] nei|e 1 nci|e 1 nc|e|a) (ek|ilâve|ilave)"

    try:
      sira_no = str(int(iter_pattern(row, pattern, 1))) + " ek 1"#int(re.search(pattern, row).group(2))
    except Exception as e:
      pattern = "(\d+) (ek|e+k|e|a) (\d+)"
      try:
        sira_no = str(int(iter_pattern(row, pattern, 1))) + " ek " + str(int(iter_pattern(row, pattern, 3)))

      except Exception as e:
        pattern = "(s\.|sira) saisi (\d+)"
        try:
          sira_no = str(int(iter_pattern(row, pattern, 2)))

        except Exception as e:
          sira_no = 0

        else:
          pass

      else:
        pass

    return sira_no

In [ ]:
#del df["pred__rega_no"]

df["pred__rega_no"] = df["data_text"].str.lower()\
                                     .str.replace("\n+", "***")\
                                     .str.replace("\s+", "")\
                                     .str.replace("sayi", "sayı")\
                                     .str.replace("sayısı", "sayı")\
                                     .str.replace("sayılı", "sayı")\
                                     .str.replace("sayıfa", "sayfa")\
                                     .apply(extract_rega_no_pattern)

df["rega_no"] = df["rega_no"].astype(str).str.replace(" \(Mükerrer\)", "").astype(float)
df["pred__rega_no"] = df["pred__rega_no"].fillna("").astype(str).str.split(".").str.get(0).str.replace(".", "").str.slice(0, 5).replace("", np.nan).astype(float)

In [ ]:
#del df["pred__mukerrer_no"] 

df.loc[~df["kategori"].isin(exclude_conf["mukerrer_no"]),
        "pred__mukerrer_no"] = df.loc[~df["kategori"].isin(exclude_conf["mukerrer_no"]),
        "data_text"].str.lower()\
                    .str.replace("\s+", " ")\
                    .apply(extract_mukerrer_no_pattern)
df.loc[(~df["data_text"].str.lower().str.replace("\s+", " ").str.contains("(\d+)\. mükerrer")) &\
       (df["data_text"].str.lower().str.split('\n').str.get(0).str.contains("mükerrer")), "pred__mukerrer_no"] = 1

In [568]:
#!pip install unidecode
#import unidecode
#df.data_text = df.data_text.apply(unidecode.unidecode).str.lower()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 235 kB 4.2 MB/s 


In [812]:
#del df["pred__sira_no"]
                    #.str.replace("\|", "1")\

df.loc[~df["kategori"].isin(exclude_conf["sira_no"]),
        "pred__sira_no"] = df.loc[~df["kategori"].isin(exclude_conf["sira_no"]),
        "data_text"].str.lower()\
                    .str.replace("'", " ")\
                    .str.replace("©", "ek")\
                    .str.replace("\(c\)", "ek")\
                    .str.replace("ek", " ek")\
                    .str.replace(":", "")\
                    .replace({"birinci": "1",
                                "ikinci": "2",
                                "ücüncü": "3",
                                "üçüncü": "3",
                                "dördüncü": "4",
                                "besinci": "5",
                                "beşinci": "5",
                                "altıncı": "6",
                                "yedinci": "7",
                                "sekizinci": "8",
                                "dokuzuncu": "9",
                                'birinci': '1',
                                'ikinci': '2',
                                'ucuncu': '3',
                                'uçuncu': '3',
                                'dörduncu': '4',
                                'besinci': '5',
                                'beşinci': '5',
                                'altıncı': '6',
                                'yedinci': '7',
                                'sekizinci': '8',
                                'dokuzuncu': '9'
                                }, regex=True)\
                    .str.replace("\s+", " ")\
                    .str.replace("s ira", "sira")\
                    .str.replace("slra", "sira")\
                    .str.replace("ı", "i")\
                    .str.replace("y", "")\
                    .apply(extract_sira_no_pattern)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: FutureWarning: The default value of regex will change from True to False in a future version.


In [813]:
label = "rega_no"
df[~df["kategori"].isin(exclude_conf[label])].assign(is_matched=lambda df: df[f"pred__{label}"] == df[label].astype(float)).aggregate(**{"is_matched": ("is_matched", "sum"),
                                                                                                                                "size": ("is_matched", "size"),
                                                                                                                                 })

,is_matched
is_matched,2478
size,2507


In [815]:
label = "sira_no"
df[~df["kategori"].isin(exclude_conf[label])].assign(is_matched=lambda df: df[f"pred__{label}"] == df[label]).aggregate(**{"is_matched": ("is_matched", "sum"),
                                                                                                                                "size": ("is_matched", "size"),
                                                                                                                                 })

,is_matched
is_matched,474
size,500


In [816]:
label = "mukerrer_no"
df[~df["kategori"].isin(exclude_conf[label])].assign(is_matched=lambda df: df[f"pred__{label}"] == df[label]).aggregate(**{"is_matched": ("is_matched", "sum"),
                                                                                                                                "size": ("is_matched", "size"),
                                                                                                                                 })

,is_matched
is_matched,2477
size,2507


In [791]:
#1 sorunu
#sirano -> sirano 
#slra -> sira
#sirasaisi -> sira saisi
#s aisi 
idx = 6

t1 = df.loc[~df["kategori"].isin(exclude_conf["sira_no"])].reset_index(drop=True)
t1 = t1[~(t1["sira_no"] == t1["pred__sira_no"])].reset_index(drop=True)
print(t1.loc[idx, "sira_no"])
print(t1.loc[idx, "pred__sira_no"])
print(t1.loc[[idx], "data_text"].str.lower()\
                    .str.replace("'", " ")\
                    .str.replace("©", "ek")\
                    .str.replace("ek", " ek")\
                    .str.replace("\s+", " ")\
                    .str.replace("s ira", "sira")\
                    .str.replace("ı", "i")\
                    .str.replace(":", "")\
                    .str.replace("y", "")[idx])
print(t1.loc[idx, "data_text"])

In [818]:
#df.loc[~df["kategori"].isin(exclude_conf["mukerrer_no"]), ["mukerrer_no", "pred__mukerrer_no"]].assign(is_matched=lambda df: df["mukerrer_no"] == df["pred__mukerrer_no"]).groupby(["mukerrer_no", "is_matched"]).size()

In [820]:
#2478/2507, 474/500, 2477/2507